<a href="https://colab.research.google.com/github/kellykryoung/AI_Study/blob/main/etc_practice/kaggle_catdog_CNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# date: 2023/10/22
# data : https://www.kaggle.com/datasets/tongpython/cat-and-dog/?select=test_set
# model : CNN model
# purpose : classification between a cat or a dog (the number of labels : 2 )
# libarary : pytorch
# code reference: https://wikidocs.net/63565
# mentor : https://github.com/shiny0510

In [1]:
import os
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torch import nn, optim
from torchvision import transforms
from torchvision.transforms import ToPILImage
import torchvision.transforms as T

from PIL import Image

import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Random seed
torch.manual_seed(777)

# Random seed in GPU
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# Hyper-parameters
learning_rate = 0.001
training_epochs =15
batch_size = 100

### Data

In [ ]:
!unzip '/content/training_set.zip'

In [ ]:
!unzip '/content/test_set.zip'

In [5]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = []


        def is_image_file(file):
            return file.endswith(('.jpg', '.jpeg', '.png', '.gif'))

        for label in range(len(self.classes)):

            class_folder = os.path.join(root_dir, self.classes[label])
            for filename in os.listdir(class_folder):
                if is_image_file(filename):
                    try:
                        img_path = os.path.join(class_folder, filename)
                        self.data.append((img_path, label))
                    except Exception as e:
                        print(f"Error processing {filename}: {e}")
                else:
                    print(f"Skipping non-image file: {filename}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label


In [6]:
data_dir = "." #현재 작업 디렉토리로 설정
batch_size = 32

transform = T.Compose([T.Resize((224,224)),
                       T.RandomRotation(5),
                       T.RandomHorizontalFlip(),
                           T.ToTensor(),
                           T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                           ])

# 01_dataset을 directory로 정의하기 (class 이용)
train_dataset = CustomDataset(os.path.join(data_dir, 'training_set'), transform=transform)
test_dataset = CustomDataset(os.path.join(data_dir, 'test_set'), transform=transform)

# 02_dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


### Model

In [10]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # the First Layer
        # ImgIn shape=(?, 224, 224, 3)
        #    Conv     -> (?, 224, 224, 32)
        #    Pool     -> (?, 112, 112, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # the Second Layer
        # ImgIn shape=(?, 112, 112, 32)
        #    Conv      ->(?, 112, 112, 64)
        #    Pool      ->(?, 56, 56, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # the Third Layer
        # ImgIn shape=(?, 56, 56, 64)
        #    Conv      ->(?, 56, 56, 128)
        #    Pool      ->(?, 28, 28, 128)

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # the Fourth Layer
        # ImgIn shape=(?, 28, 28, 128)
        #    Conv      ->(?, 28, 28, 256)
        #    Pool      ->(?, 14, 14, 256)

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # the Fifth Layer
        # ImgIn shape=(?, 14, 14, 256)
        #    Conv      ->(?, 14, 14, 128)
        #    Pool      ->(?, 7, 7, 128)
        self.layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))


        # Fully-connected Layer 7x7x128 inputs -> 2 outputs
        self.fc = torch.nn.Linear(7 * 7 * 128, 2, bias=True)

        # Initialized weight
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0), -1)   #  Flatten
        out = self.fc(out)
        return out

In [ ]:
# model class
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
cnn_model = CNN().to(device)
# samexpression
# cnn_model = CNN()
# cnn_model.to(device)

### Optimizer

In [12]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(cnn_model.parameters(), lr=0.001, momentum=0.9)

### 'for' sentence for training

In [13]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 251


In [ ]:
num_epochs = 10
best_loss = float('inf')  # Initialize the best loss as positive infinity
patience = 3  # Number of epochs to wait before early stopping
early_stop_counter = 0

for epoch in tqdm(range(num_epochs)):
    cnn_model.train()

    running_loss = 0.0

    for images, labels in train_loader:

        try:
            images, labels = images.to(device), labels.to(device)

            hypothesis = cnn_model(images)
            loss = criterion(hypothesis, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        except Exception as e:
                print(f"Error processing {images}:{e}")


    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")

     # Check for early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        early_stop_counter = 0
        # Save the best model checkpoint
        torch.save(cnn_model.state_dict(), 'best_model.pth')
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print("Early stopping. Training stopped.")
        break

# Load the best model checkpoint
cnn_model.load_state_dict(torch.load('best_model.pth'))

torch.save(cnn_model.state_dict(), 'cnn_model.pth')

### Evaluation

In [ ]:
cnn_model.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = cnn_model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy}%')